# Question-1

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# Load IMDB dataset
dataset = load_dataset("imdb")
train_dataset = dataset["train"]
test_dataset = dataset["test"]

# Define metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

def train_and_evaluate(model_name):
    print(f"\n🚀 Training model: {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

    def tokenize(batch):
        return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=256)

    tokenized_train = train_dataset.map(tokenize, batched=True)
    tokenized_test = test_dataset.map(tokenize, batched=True)

    training_args = TrainingArguments(
        output_dir=f"./results_{model_name}",
        evaluation_strategy="epoch",
        save_strategy="no",
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=1,
        logging_dir="./logs",
        report_to="none"
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train.select(range(2000)),  # subset for speed
        eval_dataset=tokenized_test.select(range(1000)),
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )

    trainer.train()
    eval_result = trainer.evaluate()
    return eval_result["eval_accuracy"]

results = {}
for model_name in ["distilbert-base-uncased", "bert-base-uncased"]:
    results[model_name] = train_and_evaluate(model_name)

best_model = max(results, key=results.get)
print("\n📊 Model Comparison Results:")
for m, acc in results.items():
    print(f"{m}: {acc:.4f}")

print(f"\n✅ Best Model: {best_model} with accuracy {results[best_model]:.4f}")



🚀 Training model: distilbert-base-uncased


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 25000/25000 [00:05<00:00, 4830.40 examples/s]


Epoch,Training Loss,Validation Loss


# Question-2

In [ ]:
!pip install -q transformers torch

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load pre-trained GPT model and tokenizer
model_name = "gpt2"  
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Example prompt
prompt = "In a distant future, humanity has discovered"

# Encode the input prompt
input_ids = tokenizer.encode(prompt, return_tensors="pt")

# Generate text
output = model.generate(
    input_ids,
    max_length=100,          # total length of generated text
    num_return_sequences=1,  # number of generated stories
    temperature=0.8,         # creativity control (0.7–1.0 = more creative)
    top_p=0.9,               # nucleus sampling
    do_sample=True,          # enables random sampling
    repetition_penalty=1.2,  # discourages repeating phrases
)

# Decode and print the generated story
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("\n🪐 Generated Story:\n")
print(generated_text)
